In [23]:
import pandas as pd
import numpy as np

# Ecommerce Dataset
There are 6 fields in the ecommerce dataset in this dataset and there will need to be some preprocessing to get the data ready for analysis.

#### is_mobile - binary variable telling us if the user is using a mobile device
#### N_products_viewed - Continuous, integer, how many products user viewed
#### Visit_duration - How long was the visitor on the site, continuous integer
#### Is_returning_visitor - binary variable marking returning visitors
#### Time_of_day - integer, 4 categories splitting the 24 hour day evenly
#### User_action - options: Bounce, add to cart, begin checkout, finish checkout

For our time variable the categories are as follows:
* 0 = 12 AM to 6 AM
* 1 = 6 AM to 12 PM
* 2 = 12 PM to 6 PM
* 3 = 6 PM to 12 AM

For the User action the categories are as follows:
* Bounce - the user visited the site and did not do anything
* Add to cart - the user added item to cart but never checked out
* Begin Checkout - The user went to start checking out but never finished
* Finish Checkout - the user finished the checkout process for their order

In [24]:
data = pd.read_csv(r'/Users/prof/Documents/machine_learning_examples/ann_logistic_extra/ecommerce_data.csv')

In [25]:
data.head()

,is_mobile,n_products_viewed,visit_duration,is_returning_visitor,time_of_day,user_action
0,1,0,0.657510,0,3,0
1,1,1,0.568571,0,2,1
2,1,0,0.042246,1,1,0
3,1,1,1.659793,1,1,2
4,0,1,2.014745,1,1,2


## Logistic Regression Example

In [26]:
N = 100
D = 2

x = np.random.randn(N,D) #Creating a normally distributed data matrix
ones = np.array([[1]*N]).T #Adding a column of 1 to add in the bias term
xb = np.concatenate((ones,x), axis = 1)

#Randomly Initialize a weight vector to show how to use the sigmoid
w = np.random.randn(D + 1)

In [27]:
#Multiplying our weights and independent variables using numpy dot function
z = xb.dot(w)

In [28]:
#Creating the sigmoid function
def sigmoid(z):
    return 1/(1+np.exp(z))

In [29]:
sigmoid(z)

array([0.69456568, 0.67281618, 0.65251772, 0.72145095, 0.57746062,
       0.7092308 , 0.52297776, 0.48668909, 0.64916322, 0.85425149,
       0.57211232, 0.66044608, 0.74619792, 0.85743489, 0.67683743,
       0.74534232, 0.48235673, 0.69852963, 0.49532346, 0.77186857,
       0.67943208, 0.63567432, 0.65390778, 0.56511785, 0.44467205,
       0.73306633, 0.6444764 , 0.71585288, 0.790717  , 0.58383917,
       0.77303055, 0.68044869, 0.59357796, 0.7863176 , 0.65662495,
       0.52012304, 0.66395822, 0.71741855, 0.73897693, 0.74345857,
       0.5432722 , 0.70990199, 0.73838227, 0.59758029, 0.72753616,
       0.79769935, 0.67361625, 0.74259057, 0.65675113, 0.6797007 ,
       0.67331348, 0.5989304 , 0.64005053, 0.72705956, 0.57454453,
       0.6975067 , 0.67266616, 0.64308499, 0.60033897, 0.48602566,
       0.59077885, 0.82013846, 0.71636659, 0.79194221, 0.77940937,
       0.66004083, 0.64938355, 0.64741364, 0.52275008, 0.76268346,
       0.66046513, 0.52952064, 0.73946078, 0.72691537, 0.72663